In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

notebooks_dir = Path.cwd() if Path.cwd().name == 'notebooks' else Path.cwd() / 'notebooks'
output_dir = notebooks_dir / 'output'
output_dir.mkdir(parents=True, exist_ok=True)
temp_db = output_dir / 'temp_database.db'
if temp_db.exists():
    temp_db.unlink()
conn = sqlite3.connect(str(temp_db))
with open(notebooks_dir / 'database.sql', 'r', encoding='utf-8') as f:
    conn.executescript(f.read())
conn.commit()

In [2]:
judge_counts = pd.read_sql_query("""
    SELECT event_id, COUNT(DISTINCT judge_id) AS judge_count FROM event_judges GROUP BY event_id
""", conn)
judge_map = dict(zip(judge_counts['event_id'], judge_counts['judge_count']))

In [3]:
reg_scores = pd.read_sql_query("""
    SELECT r.id AS registration_id, r.event_id, COALESCE(SUM(j.score), 0) AS total_score
    FROM registrations r
    INNER JOIN judgments j ON r.id = j.registration_id
    GROUP BY r.id, r.event_id
    HAVING total_score > 0
""", conn)
reg_scores['normalized_score'] = reg_scores.apply(
    lambda r: (r['total_score'] / (judge_map.get(r['event_id'], 1) * 50)) * 100 if judge_map.get(r['event_id'], 1) * 50 > 0 else 0,
    axis=1
).round(1)

In [4]:
student_participation = pd.read_sql_query("""
    SELECT registration_id, participant_id AS student_id
    FROM registration_participants WHERE participant_type = 'student'
""", conn)
student_scores = student_participation.merge(
    reg_scores[['registration_id', 'normalized_score']], on='registration_id', how='inner'
)
total_per_student = student_scores.groupby('student_id')['normalized_score'].sum().reset_index()
total_per_student.columns = ['student_id', 'total_normalized_score']
total_per_student['total_normalized_score'] = total_per_student['total_normalized_score'].round(1)

In [5]:
students_df = pd.read_sql_query("SELECT id, student_id AS student_roll, student_name, gender, school_id FROM students", conn)
schools_df = pd.read_sql_query("SELECT id, name AS school_name, school_code FROM schools", conn)
ranked = (
    total_per_student.merge(students_df, left_on='student_id', right_on='id')
    .merge(schools_df, left_on='school_id', right_on='id')
)[['student_id', 'student_roll', 'student_name', 'gender', 'school_name', 'school_code', 'total_normalized_score']]
ranked = ranked.sort_values('total_normalized_score', ascending=False).reset_index(drop=True)

male_candidates = ranked[ranked['gender'] == 'male']
female_candidates = ranked[ranked['gender'] == 'female']
top_male = male_candidates.iloc[0] if len(male_candidates) > 0 else None
top_female = female_candidates.iloc[0] if len(female_candidates) > 0 else None

In [6]:
reg_with_events = pd.read_sql_query("""
    SELECT r.id AS registration_id, r.event_id, r.chest_number, r.team_name,
           e.name AS event_name, e.event_type, e.age_category
    FROM registrations r JOIN events e ON r.event_id = e.id
""", conn)
reg_scores_with_detail = reg_scores.merge(reg_with_events, on=['registration_id', 'event_id'])
reg_scores_with_detail['judge_count'] = reg_scores_with_detail['event_id'].map(judge_map)
reg_scores_with_detail['max_possible'] = reg_scores_with_detail['judge_count'] * 50

events_and_results_per_student = student_participation.merge(
    reg_scores_with_detail[[
        'registration_id', 'event_name', 'event_type', 'age_category',
        'chest_number', 'team_name', 'total_score', 'judge_count', 'max_possible', 'normalized_score'
    ]], on='registration_id', how='inner'
).rename(columns={'normalized_score': 'normalized_score_0_100'})

In [ ]:
def show_events_and_results(label, candidate, events_df):
    print("=" * 60)
    print(label)
    print("=" * 60)
    if candidate is None:
        print("  No candidates with scores.\n")
        return
    print(f"  Name:   {candidate['student_name']}  |  School: {candidate['school_name']} ({candidate['school_code']})")
    print(f"  Total (sum of normalized): {candidate['total_normalized_score']}\n  Events and results:")
    df = events_df[events_df['student_id'] == candidate['student_id']]
    for _, row in df.iterrows():
        team = f"  [Team: {row['team_name']}]" if pd.notna(row['team_name']) and row['team_name'] else ""
        print(f"  • {row['event_name']} ({row['event_type']}, {row['age_category']}){team}")
        print(f"      Raw {row['total_score']}/{row['max_possible']} (judges: {row['judge_count']}) → Normalized: {row['normalized_score_0_100']}")
    print()

show_events_and_results("HIGHEST SCORING MALE", top_male, events_and_results_per_student)
show_events_and_results("HIGHEST SCORING FEMALE", top_female, events_and_results_per_student)

cols = ['event_name', 'event_type', 'age_category', 'chest_number', 'team_name', 'total_score', 'max_possible', 'judge_count', 'normalized_score_0_100']
if top_male is not None:
    display(events_and_results_per_student[events_and_results_per_student['student_id'] == top_male['student_id']][cols])
if top_female is not None:
    display(events_and_results_per_student[events_and_results_per_student['student_id'] == top_female['student_id']][cols])

In [ ]:
# Top 5 male and female
print("Top 5 male:")
display(male_candidates.head() if len(male_candidates) > 0 else pd.DataFrame())
print("Top 5 female:")
display(female_candidates.head() if len(female_candidates) > 0 else pd.DataFrame())

In [9]:
conn.close()